In [7]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
     ASK {
            <http://dbpedia.org/resource/Businessperson> <http://dbpedia.org/resource/Type_species> <http://dbpedia.org/resource/Model_(person)>.
        }   
""")

print()
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)


{'head': {'link': []}, 'boolean': False}


In [3]:
import csv
import re
import spacy
from spacy import displacy

import urllib
#from owlready2 import *
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON

import re
import xml.etree.ElementTree as ET

from allennlp.common.testing import AllenNlpTestCase
from allennlp.predictors.predictor import Predictor

# pre-processing
def PreProcess(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")
            
# stopwords from parsing the whole sentence
def RemoveStopword1(phrase, doc, chunkStart, chunkEnd, stopList):
    result = phrase
    i_stop=0
    #start = chunk.start# to eliminate the condition when the first word of chunk is stop word
    for i_sen in range(chunkStart, chunkEnd):
        while i_stop < len(stopList) and stopList[i_stop] < i_sen-1:
            #print(str(stopList[i_stop]) + ' ' + str(i_sen))
            i_stop = i_stop+1
        # there is no stop word in current chunk
        if i_stop >= len(stopList):
            break;
        #print(i_sen)
        # finish going through the chunk
        if stopList[i_stop] > chunkEnd-1:
            break
        # find the stop word and remove it
        if stopList[i_stop] == i_sen-1:
            #print(doc[i_sen-1])
            if i_sen-1 == chunkStart:
                result = result.replace(doc[i_sen-1].text + ' ', '')
                chunkStart = chunkStart+1
            else:
                result = result.replace(' ' + doc[i_sen-1].text, '')
    return result

# stopwords from parsing triple separately
def RemoveStopword2(inputPhrase):
    result = ''
    doc_phrase = nlp(str(inputPhrase))
    for token in doc_phrase:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #       token.shape_, token.is_alpha, token.is_stop)
        if not token.is_stop:
            result = result + token.text + ' '
        #else:
        #    print(token.text + ', ', end = '')    
    return result


# extract one triple from given sentence
def ExtractTriple(sen):
    # initialize the triple and stop word list
    subj = ""
    pred = ""
    obj = ""
    stopList = []
    
    # parse sentence
    doc = nlp(str(sen))
    print('\n' + str(index) + '. Original Sentence:\n' + sen)
    
    ## visualize the semantic tree
    #options = {'compact': True, 'color': 'blue'}
    #displacy.serve(doc, style='dep', options=options)
    #displacy.serve(doc, style='dep')

    print('\nStopwords:')
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #      token.shape_, token.is_alpha, token.is_stop)

        # record the index of stop words
        if token.is_stop:
            print(token.text + ', ', end='')
            stopList.append(token.i)
        if re.match('nsubj', token.dep_):   
            subj = token.text
        if re.match('ROOT', token.dep_): 
            pred = token.lemma_
            pred_orig = token.text
        if re.match('dobj', token.dep_): 
            obj = token.text
            '''#an earlier solution that I find not necessary
            obj = token.lemma_
            # to avoid cases like "-PRON-"
            if obj[0] == '-':
                obj = token.text'''
    print('\n')

    subj_1 = subj
    obj_1 = obj
    # using chunk to update subject and object
    for chunk in doc.noun_chunks:
        if chunk.root.head.text == pred_orig and re.match('nsubj', chunk.root.dep_):
            subj = chunk.text
            # remove stop words
            subj_1 = RemoveStopword1(subj, doc, chunk.start, chunk.end, stopList)

        if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
            obj = chunk.text
            # remove stop words
            obj_1 = RemoveStopword1(obj, doc, chunk.start, chunk.end, stopList)
        #print(chunk.text + ' ' + str(chunk.start))
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

    #print('Before : ' + subj + ' - ' + pred + ' - ' + obj)
    #print('Method1: ' + subj_1 + ' - ' + pred + ' - ' + obj_1)

    # second method to remove stop words
    subj_2 = RemoveStopword2(subj)
    obj_2 = RemoveStopword2(obj)
    #print('Method2: ' + subj_2 + '- ' + pred + ' - ' + obj_2 + '\n')

    return [subj, pred, obj]

def QueryURI(keywords, index=-2):
    localSite = 'http://localhost:1111/api/search/KeywordSearch?'
    onlineSite = 'http://lookup.dbpedia.org/api/search/KeywordSearch?'
    prefix = "{http://lookup.dbpedia.org/}"
    
    keywords = keywords.replace(' ', "%20")
    request = onlineSite + \
    'QueryClass='   + ''  + \
    '&MaxHits='     + '5' + \
    '&QueryString=' + keywords
    response = str(urllib.request.urlopen(request).read(), 'utf-8')

    root = ET.fromstring(response)
    result = root.findall(prefix + "Result")
    
    if len(result)>0:
        selected = -1
        count = 0
        for name in result:
            print(str(count) + ": " + name.find(prefix + "Label").text)
            count += 1
        # for some default input during debugging
        if index<-1:
            index = int(input("Which one is closer to what you mean? (type \"-1\" if nothing seems correct) "))
        if index >= 0:
            selected = "<" + result[index].find(prefix + "URI").text + ">"
        else:
            selected = None
        return selected
    else:
        print("Sorry, we find nothing for this stuff :(\n")
        return None

# transfer a phrase to a URI form
def FormatURI(phrase, isS_O = False):
    #print('Before formatting:  ' + phrase)
    chars = list(phrase)
    
    if len(chars) > 0 and not isS_O:
        chars[0] = chars[0].upper()
    for i in range(len(chars)):
        if chars[i] == ' ' and i+1 < len(chars):
            chars[i+1] = chars[i+1].upper()
    phrase = ''.join(chars)
    phrase = phrase.replace(' ', '')
    phrase = re.sub(r'[^a-zA-Z0-9\s]', '', phrase)
    #print('After formatting:  ' + phrase)
    return phrase

# transfer a phrase to a URI form
def NameURI(url):
    index_slash = 0
    for i in range(len(url)-1, -1, -1):
        if url[i] == '/':
            index_slash = i+1
            break
    return url[index_slash:]

# query the given triple in the ontology with SPARQL
# return true/false as result
def QueryTriple(subj, pred, obj):
    if subj==None or pred==None or obj==None:
        return None
    else:
        prefix = """
        PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbpd:<http://dbpedia.org/ontology/>
        """
        #subj = "provinceLink"
        #pred = "range"
        #obj = "Province"
        qSelect = prefix + """
        SELECT ?sub WHERE {
          ?sub rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
        }"""
        qAsk = prefix + """
        ASK {
            dbpd:""" + FormatURI(subj) + """ rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
        }"""

        r = list(m_graph.query(qAsk))
        return r

def ComponentQuery1(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "province"
    
    r = []
    if subj!=None:
        qSelect_P_O = prefix + """
        SELECT ?pred ?obj WHERE {
          """ + subj + """ ?pred ?obj.
        }"""
        #r1 = m_graph.query(qSelect_P_O)
        sparql.setQuery(qSelect_P_O)
        r1 = sparql.query().convert()
        if r1 != None: # may need one more variable to record source
            r.append(r1)
        
    if pred!=None:
        qSelect_S_O = prefix + """
        SELECT ?sub ?obj WHERE {
          ?sub """ + pred + """ ?obj.
        }"""
        #r2 = m_graph.query(qSelect_S_O) 
        sparql.setQuery(qSelect_S_O)
        r2 = sparql.query().convert()
        if r2 != None: # may need one more variable to record source
            r.append(r2)
        
    if obj!=None:
        qSelect_S_P = prefix + """
        SELECT ?sub ?pred WHERE {
          ?sub ?pred """ + obj + """.
        }"""
        #r3 = m_graph.query(qSelect_S_P) 
        sparql.setQuery(qSelect_S_P)
        r3 = sparql.query().convert()
        if r3 != None: # may need one more variable to record source
            r.append(r3)

    #if r!=[]:
    #    print(r)
    return r

def ComponentQuery2(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """    
    
    r = []
    if pred!=None and obj!=None:
        qSelect_S = prefix + """
        SELECT ?sub WHERE {
          ?sub """ + pred + """ """ + obj + """.
        }"""
        
        #r1 = m_graph.query(qSelect_S) 
        sparql.setQuery(qSelect_S)
        r1 = sparql.query().convert()
        if r1 != None: # may need one more variable to record source
            r.append(r1)
        
    if subj!=None and obj!=None:
        qSelect_P = prefix + """
        SELECT ?pred WHERE {
          """ + subj + """ ?pred """ + obj + """.
        }"""
        
        #r2 = m_graph.query(qSelect_P)
        sparql.setQuery(qSelect_P)
        r2 = sparql.query().convert()
        if r2 != None: # may need one more variable to record source
            r.append(r2)
            
    if subj!=None and pred!=None:
        qSelect_O = prefix + """
        SELECT ?obj WHERE {
          """ + subj + """ """ + pred + """ ?obj.
        }"""
        
        #r3 = m_graph.query(qSelect_O)
        sparql.setQuery(qSelect_O)
        r3 = sparql.query().convert()
        if r3 != None: # may need one more variable to record source
            r.append(r3)
        
    #if r!=[]:
    #    print(r)
    #PrintQueryResult(r)
    return r

def ComponentQuery3(subj, pred, obj):
    if subj==None or pred==None or obj==None:
        return None
    else:
        prefix = """
        PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbpd:<http://dbpedia.org/ontology/>
        """
        #subj = "provinceLink"
        #pred = "range"
        #obj = "province"

        qAsk = prefix + """
        ASK WHERE {
            """ + subj + """ """ + pred + """ """ + obj + """
        }"""
        #print(qAsk)

        #r = m_graph.query(qAsk)
        sparql.setQuery(qAsk)
        sparql.setReturnFormat(JSON)
        r = sparql.query().convert()

        #if not r:
        #    print(qAsk)
        #    print(r)
        
        return r["boolean"]

def PartialQuery(subj, pred, obj, subjURI, predURI, objURI):
    if subj==None and pred==None and obj==None:
        return None
    
    print("\n*********************** In Partial Query *************************")
    doc_subj = nlp(str(subj))
    doc_pred = nlp(str(pred))
    doc_obj = nlp(str(obj))
    r1 = []
    r2 = []
    r3 = False
    
    for token_subj in doc_subj:
        # get the URI for partial subj
        if len(doc_subj)>1:
            print("\nFor partial subject \"" + token_subj.text + "\":")
            part_subj = QueryURI(token_subj.text)
        else:
            part_subj = subjURI
            
        for token_pred in doc_pred:
            # get the URI for partial pred
            if len(doc_pred)>1:
                print("\nFor partial predicate \"" + token_pred.text + "\":")
                part_pred = QueryURI(token_pred.text)
            else:
                part_pred = predURI
                
            for token_obj in doc_obj:
                #print(token_obj.text, token_obj.lemma_, token_obj.pos_, token_obj.tag_, token_obj.dep_,
                #      token_obj.shape_, token_obj.is_alpha, token_obj.is_stop)
                if token_subj.is_stop and token_pred.is_stop and token_obj.is_stop:
                    continue
                    
                # get the URI for partial obj
                if len(doc_obj)>1:
                    print("\nFor partial object \"" + token_obj.text + "\":")
                    part_obj = QueryURI(token_obj.text)
                else:
                    part_obj = objURI

                r1 = ComponentQuery1(part_subj, part_pred, part_obj)
                #print(r1)
                if r1 and len(r1)>0:
                    r2 = ComponentQuery2(part_subj, part_pred, part_obj)
                    r2Result = False
                    for group in r2:
                        for result in group["results"]["bindings"]:
                            if len(result) > 0:
                                r2Result = True
                    if r2Result:
                        r3 = ComponentQuery3(part_subj, part_pred, part_obj)
                        if r3:
                            print("\nFind triple with 3 partial components:")
                            print(part_subj + " - " + part_pred + " - " + part_obj)
                            break
                        else:
                            print("\nFind triple with 2 partial components")
                            PrintQueryResult(r2, part_subj, part_pred, part_obj)
                            #print(r2)
                    else:
                        print("\nFind single partial component(s):")
                        if part_subj != None:
                            print(part_subj[1:len(part_subj)-1])
                        if part_pred != None:
                            print(part_pred[1:len(part_pred)-1])
                        if part_obj != None:
                            print(part_obj[1:len(part_obj)-1])
                        PrintQueryResult(r1, part_subj, part_pred, part_obj)
    
    return True
    #return len(r1)>0 or r2Result or r3

def PrintQueryResult(results, sub, pred, obj):
    # for sparqlWrapper
    for group in results:
        #print(group)
        for result in group["results"]["bindings"]:
            print('( ', end='')
            if "sub" in result:
                print(NameURI(result["sub"]["value"]) + ' - ', end='')
            else:
                if '<' in sub:
                    sub = sub[29:-1]
                print(sub + ' -', end='')
            if "pred" in result:
                print(NameURI(result["pred"]["value"]) + ' - ', end='')
            else:
                if '<' in pred:
                    pred = pred[29:-1]
                print(pred + ' -', end='')
            if "obj" in result:
                print(NameURI(result["obj"]["value"]) + ' )\n', end='')
            else:
                if '<' in obj:
                    obj = obj[29:-1]
                print(obj + ' )\n', end='')
        
    '''# for owlready
    for result in results:
        for var in result.vars:
            print(var.toPython())
        for binding in result.bindings:
            print(binding.toPython())'''
        
# load Spacy NLP dictionary
nlp = spacy.load('en_core_web_sm')

# load DBPD ontology and construct graph for query
#m_world = World()# Owlready2 stores every triples in a ‘World’ object
#m_onto = m_world.get_ontology("dbpedia.owl").load()
#m_graph = m_world.as_rdflib_graph()
#sparql = SPARQLWrapper("dbpedia.owl")#http://dbpedia.org/sparql
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

# load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)
senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array
file.close()
print("Total sentences: " + str(len(senSet)))

# pre-processing
PreProcess(senSet)

# parse and query each sentence
#for index in range(len(senSet)):
index = 26
sampleSentence = "Businessman is a person."
#sampleSentence = "Neverland has the tree house."

# extract triple from current sentence
[subj, pred, obj] = ExtractTriple(sampleSentence)
#[subj, pred, obj] = ExtractTriple(senSet[index])
print('Triple to Query: \n' + subj + ' - ' + pred + ' - ' + obj)

# parse with AllenNLP
'''from allennlp.predictors import Predictor
predictor = Predictor.from_path("srl-model-2018.05.25.tar.gz")
results = predictor.predict(senSet[index])
for verb in zip(results["verbs"]):
    print(f"{verb}")
#for word, verb in zip(results["words"], results["verbs"]):
#    print(f"{word}\t{verb}")
'''
#for word, tag in zip(results["words"], results["tags"]):
#    print(f"{word}\t{tag}")

#subj = "province link"
if pred == "be":
    pred = "type"
#obj = "person"

# look up the URI for subj, pred and obj
print("\nFor subject \"" + subj + "\":")
subjURI = QueryURI(subj)
print("\nFor predicate \"" + pred + "\":")
predURI = QueryURI(pred)
print("\nFor object \"" + obj + "\":")
objURI = QueryURI(obj)

try:
    print("\n")
    print("subject: " + subjURI[1:len(subjURI)-1])
    print("predicate: " + predURI[1:len(predURI)-1])
    print("object: " + objURI[1:len(objURI)-1])
except:
    print("none")
    
# query the triple in dbpd with SPARQL
# queryResult = QueryTriple(subj, pred, obj)
# print('Triple Query Result: ' + str(queryResult))

# query the triple with several different methods
r3 = ComponentQuery3(subjURI, predURI, objURI)
if r3:
    print("\nFind origin component:")
    print(subj + ' - ' + pred + ' - ' + obj)
else:
    r2 = ComponentQuery2(subjURI, predURI, objURI)
    if r2 and len(r2)>0:
        print("\nFind 2 components:")
        PrintQueryResult(r2, subj, pred, obj)#print(r2) 
    else:
        r1 = ComponentQuery1(subjURI, predURI, objURI)
        if r1 and len(r1)>0:
            print("\nFind single component(s):")
            if subjURI != None:
                print(subjURI[1:len(subjURI)-1])
            if predURI != None:
                print(predURI[1:len(predURI)-1])
            if objURI != None:
                print(objURI[1:len(objURI)-1])
            PrintQueryResult(r1, subj, pred, obj)#print(r1) 
    
    #partial query
    result = PartialQuery(subj, pred, obj, subjURI, predURI, objURI)
    if not result:
        print ("Find nothing in parial query")

Total sentences: 46
Pre-processing...

26. Original Sentence:
Businessman is a person.

Stopwords:
is, a, 

Triple to Query: 
Businessman - be - a person

For subject "Businessman":
0: Business
1: Businessperson
2: Business magnate
3: Petroleum industry
4: Small business
Which one is closer to what you mean? (type "-1" if nothing seems correct) 1

For predicate "type":
0: Types of inhabited localities in Russia
1: Type species
2: Köppen climate classification
3: Holotype
4: Rock (geology)
Which one is closer to what you mean? (type "-1" if nothing seems correct) 1

For object "a person":
0: Model (person)
1: Japanese people
2: Businessperson
3: Chinese people
4: White people
Which one is closer to what you mean? (type "-1" if nothing seems correct) 0


subject: http://dbpedia.org/resource/Businessperson
predicate: http://dbpedia.org/resource/Type_species
object: http://dbpedia.org/resource/Model_(person)

Find 2 components:

*********************** In Partial Query ********************

0: Model (person)
1: Japanese people
2: Businessperson
3: Chinese people
4: White people
Which one is closer to what you mean? (type "-1" if nothing seems correct) 0

Find single partial component(s):
http://dbpedia.org/resource/Businessperson
http://dbpedia.org/resource/Type_species
http://dbpedia.org/resource/Model_(person)
( Businessperson -22-rdf-syntax-ns#type - owl#Thing )
( Businessperson -rdf-schema#label - Businessperson )
( Businessperson -rdf-schema#label - شخصيات من عالم الأعمال )
( Businessperson -rdf-schema#label - Empresario )
( Businessperson -rdf-schema#label - Personnalité du monde des affaires )
( Businessperson -rdf-schema#label - 実業家 )
( Businessperson -rdf-schema#label - Executivo )
( Businessperson -rdf-schema#label - 實業家 )
( Businessperson -rdf-schema#comment - 実業家（じつぎょうか）は、実業すなわち生産・流通・販売などの過程における事業を行う人物。多くの場合、企業経営者・使用者もしくは企業の幹部と同義語である。 資本家との対比で用いられることが多いが、資本家であると同時に実業家である場合もある。市場経済における重要な担い手である。 )
( Businessperson -rdf-schema#comment - Executivo é a função de q

( Poppy_Delevingne - wikiPageWikiLink - Model_(person) )
( Poster_girl - wikiPageWikiLink - Model_(person) )
( Priya_Sachdev - wikiPageWikiLink - Model_(person) )
( Rafael_Rosell - wikiPageWikiLink - Model_(person) )
( Roshni_Chopra - wikiPageWikiLink - Model_(person) )
( Sahithya_Jagannathan - wikiPageWikiLink - Model_(person) )
( Simona_Andrejić - wikiPageWikiLink - Model_(person) )
( Sunitha_Varma - wikiPageWikiLink - Model_(person) )
( Terry_Markwell - wikiPageWikiLink - Model_(person) )
( The_Tin_Drum - wikiPageWikiLink - Model_(person) )
( Thomas_Helmig - wikiPageWikiLink - Model_(person) )
( Todd_McMillon - wikiPageWikiLink - Model_(person) )
( Warrap_(state) - wikiPageWikiLink - Model_(person) )
( Devon_Aoki - wikiPageWikiLink - Model_(person) )
( Jane_Leeves - wikiPageWikiLink - Model_(person) )
( Jane_Russell - wikiPageWikiLink - Model_(person) )
( Olive_Thomas - wikiPageWikiLink - Model_(person) )
( Ramjas_College - wikiPageWikiLink - Model_(person) )
( List_of_people_from_W

( List_of_Playboy_Playmates_of_2004 - wikiPageWikiLink - Model_(person) )
( Luna_Maya - wikiPageWikiLink - Model_(person) )
( Mary_Bacon - wikiPageWikiLink - Model_(person) )
( Miguel_Rodriguez_(actor) - wikiPageWikiLink - Model_(person) )
( Nathalie_Cox - wikiPageWikiLink - Model_(person) )
( Nicole_Flint - wikiPageWikiLink - Model_(person) )
( Pietro_Taricone - wikiPageWikiLink - Model_(person) )
( Richa_Gangopadhyay - wikiPageWikiLink - Model_(person) )
( Roberta_Murgo - wikiPageWikiLink - Model_(person) )
( Shunsuke_Ikeda - wikiPageWikiLink - Model_(person) )
( Skal_vi_danse%3F_(season_5) - wikiPageWikiLink - Model_(person) )
( Svetlana_Lazareva - wikiPageWikiLink - Model_(person) )
( The_Dirk_Diggler_Story - wikiPageWikiLink - Model_(person) )
( Tika_Patsatsia - wikiPageWikiLink - Model_(person) )
( Aleksandra_Grdić - wikiPageWikiLink - Model_(person) )
( Angie_Ng - wikiPageWikiLink - Model_(person) )
( Ankita_Sharma - wikiPageWikiLink - Model_(person) )
( Chesster_Chay - wikiPage

( DJ_Qualls - wikiPageWikiLink - Model_(person) )
( Daniel_da_Cruz_Carvalho - wikiPageWikiLink - Model_(person) )
( Darla_Pacheco - wikiPageWikiLink - Model_(person) )
( Elie_Saab - wikiPageWikiLink - Model_(person) )
( Emerald_Ignacio - wikiPageWikiLink - Model_(person) )
( Geraldine_Doyle - wikiPageWikiLink - Model_(person) )
( Giorgia_Palmas - wikiPageWikiLink - Model_(person) )
( Hazel_Keech - wikiPageWikiLink - Model_(person) )
( Holly_Willoughby - wikiPageWikiLink - Model_(person) )
( Human_sexuality - wikiPageWikiLink - Model_(person) )
( Iwa_Moto - wikiPageWikiLink - Model_(person) )
( Jack_Wrangler - wikiPageWikiLink - Model_(person) )
( Jean_Paul_Gaultier - wikiPageWikiLink - Model_(person) )
( John_Estrada - wikiPageWikiLink - Model_(person) )
( Joseph_Estrada - wikiPageWikiLink - Model_(person) )
( Justine_Joli - wikiPageWikiLink - Model_(person) )
( Justine_Lévy - wikiPageWikiLink - Model_(person) )
( Karen_Mulder - wikiPageWikiLink - Model_(person) )
( Luis_Fonsi - wikiPa

...

( Sameksha - occupation - Model_(person) )
( Támar - occupation - Model_(person) )
( Ursula_Karven - occupation - Model_(person) )
( Utt_Panichkul - occupation - Model_(person) )
( Vaughn_Lowery - occupation - Model_(person) )
( Yoon_Eun-hye - occupation - Model_(person) )
( Yū_Aoi - occupation - Model_(person) )
( Maggie_McNamara - occupation - Model_(person) )
( Betty_Davis - occupation - Model_(person) )
( Kelly_Monaco - occupation - Model_(person) )
( Summer_Phoenix - occupation - Model_(person) )
( Debbe_Dunning - occupation - Model_(person) )
( Sylvia_Ashley - occupation - Model_(person) )
( Kim_Ki-bum - occupation - Model_(person) )
( Lisa_Haydon - occupation - Model_(person) )
( Maria_Fernanda_Cândido - occupation - Model_(person) )
( Sachini_Ayendra_Stanley - occupation - Model_(person) )
( Sanjeeda_Sheikh - occupation - Model_(person) )
( Àstrid_Bergès-Frisbey - occupation - Model_(person) )
( Andrea_Grant - occupation - Model_(person) )
( Ariana_Barouk - occupation - Model_(

( Carla_Abellana - occupation - Model_(person) )
( Danny_Lopes - occupation - Model_(person) )
( Empress_Schuck - occupation - Model_(person) )
( Gwendolyn_Osborne - occupation - Model_(person) )
( Ikki_Twins - occupation - Model_(person) )
( Ileana_D'Cruz - occupation - Model_(person) )
( Jenilca_Giusti - occupation - Model_(person) )
( Jolin_Chien - occupation - Model_(person) )
( Kostas_Sommer - occupation - Model_(person) )
( Lauren_Bush - occupation - Model_(person) )
( Lauren_Hutton - occupation - Model_(person) )
( Lauren_McAvoy - occupation - Model_(person) )
( Lauren_Michelle_Hill - occupation - Model_(person) )
( Mariel_Rodriguez - occupation - Model_(person) )
( Naeem_Haq - occupation - Model_(person) )
( Pooja_Hegde - occupation - Model_(person) )
( Reichen_Lehmkuhl - occupation - Model_(person) )
( Rico_Yan - occupation - Model_(person) )
( Sonia_Ferrer - occupation - Model_(person) )
( Vaishali_Desai - occupation - Model_(person) )
( Gretchen_Barretto - occupation - Model

( Beverly_Johnson - occupation - Model_(person) )
( Bridget_Marquardt - occupation - Model_(person) )
( Bridget_Moynahan - occupation - Model_(person) )
( Cynthia_Gibb - occupation - Model_(person) )
( Dewi_Sandra - occupation - Model_(person) )
( Gabrielle_Union - occupation - Model_(person) )
( Jas_Arora - occupation - Model_(person) )
( Jive_Jones - occupation - Model_(person) )
( John_Lloyd_Cruz - occupation - Model_(person) )
( Mew_Azama - occupation - Model_(person) )
( Micah_Alberti - occupation - Model_(person) )
( Queen_Latifah - occupation - Model_(person) )
( Rafael_Amaya - occupation - Model_(person) )
( Ravi_(rapper) - occupation - Model_(person) )
( Roma_Asrani - occupation - Model_(person) )
( Sana_Nawaz - occupation - Model_(person) )
( Albert_Reed - occupation - Model_(person) )
( Alexandre_Rodrigues_(actor) - occupation - Model_(person) )
( Aljur_Abrenica - occupation - Model_(person) )
( April_Scott - occupation - Model_(person) )
( Cho_Seung-hee_(entertainer) - occu

( Tooji - occupation - Model_(person) )
( Vasuki_Sunkavalli - occupation - Model_(person) )
( Yureni_Noshika - occupation - Model_(person) )
( Zaira_Nara - occupation - Model_(person) )
( Özge_Ulusoy - occupation - Model_(person) )
( Beth_Ostrosky_Stern - occupation - Model_(person) )
( Carey_Lowell - occupation - Model_(person) )
( Rebecca_de_Alba - occupation - Model_(person) )
( Sharmila_Tagore - occupation - Model_(person) )
( Adriana_Arboleda - occupation - Model_(person) )
( Frencheska_Farr - occupation - Model_(person) )
( Gee-Ann_Abrahan - occupation - Model_(person) )
( Humaima_Malick - occupation - Model_(person) )
( Johan_Santos - occupation - Model_(person) )
( Lina_Marulanda - occupation - Model_(person) )
( Maricar_Reyes - occupation - Model_(person) )
( Moulshree_Sachdeva - occupation - Model_(person) )
( Santiago_Ramundo - occupation - Model_(person) )
( Gul_Panag - occupation - Model_(person) )
( Aaron_Yan - occupation - Model_(person) )
( Ana_Álvarez - occupation - Mo

( Surbhi_Javeri_Vyas - occupation - Model_(person) )
( Surjit_Saha - occupation - Model_(person) )
( Tussaneeya_Karnsomnut - occupation - Model_(person) )
( Vin_Rana - occupation - Model_(person) )
( Waluscha_de_Sousa - occupation - Model_(person) )
( Wasim_Mushtaq - occupation - Model_(person) )
( Aalia_Furniturewalla - occupation - Model_(person) )
( Aarushi_Sharma - occupation - Model_(person) )
( Aman_Hundal - occupation - Model_(person) )
( Arjun_Nandhakumar - occupation - Model_(person) )
( Art_Supawatt_Purdy - occupation - Model_(person) )
( Aseem_Ali_Khan - occupation - Model_(person) )
( Bello_Sánchez - occupation - Model_(person) )
( Djamilya_Abdullaeva - occupation - Model_(person) )
( Jess_Impiazzi - occupation - Model_(person) )
( Jung_Chae-yeon - occupation - Model_(person) )
( Kadambari_Jethwani - occupation - Model_(person) )
( Kelly_Fu - occupation - Model_(person) )
( Kim_Domingo - occupation - Model_(person) )
( Lopamudra_Raut - occupation - Model_(person) )
( Madhuc

In [3]:
# reference: https://pythonhosted.org/Owlready2/world.html
from owlready2 import *
import rdflib

my_world = World()# Owlready2 stores every triples in a ‘World’ object
onto = my_world.get_ontology("dbpedia.owl").load()

graph = my_world.as_rdflib_graph()
print(len(graph))

prefix = """
PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbpd:<http://dbpedia.org/ontology/>
"""

'''r = list(graph.query(prefix + """
SELECT ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))'''

'''r = list(graph.query(prefix + """
ASK {
  dbpd:provinceLink rdf:range  dbpd:Province.
}"""))'''

r = list(graph.query(prefix + """
DESCRIBE ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))

print(r)

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataPrope

31050


Exception: DESCRIBE not implemented

In [3]:
def revlist(alist):
    if not alist:
        return []
    return [alist[-1]] + revlist(alist[:-1])

revlist(['cat', 42, 'dog', 3.14159])

[3.14159, 'dog', 42, 'cat']